
GTFS Wiki: http://wiki.corp.nextbus.com/nextbuswiki/GtfsChecks (need to be on NexBus VPN) 

GTFS Goolge Guideline: https://developers.google.com/transit/gtfs/reference/

# <span style="color:red">Import the needed packages</span>

In [1]:
import pandas as pd
import numpy as np
#import pixiedust as pix # will look into the capabilites of this library

### <span style="color:blue">Add the location of your GTFS data</span>

In [18]:
# for windows
# filepath = r"\\cts-con-fld-001\users$\207329\My Documents\qa\vacaville\vacavillecitycoach-ca-us_20181015"

# for linux
# filepath = r"../../../mnt/shared/qa/omnitrans"


#filepath2 = r"\\cts-con-fld-001\users$\207329\My Documents\qa\sf-muni\sf-muni_export_20180907"

'''Some functions to make life eaiser'''

'''write a funtion that queries out a specific GTFS schedule from a larger schedule- think seattle-sc'''

# load file and show top 5 records
def load(fName): 
    f = filepath + "\\" + "{}.txt".format(fName)
    global df
    df = pd.read_csv(f)
    return df.info()

# load second file for comparision
def load2(fName): 
    f = filepath2 + "\\" + "{}.txt".format(fName)
    global df2
    df2 = pd.read_csv(f)

import csv
# short save file
def save(dfName,fName):
    df = dfName
    df.to_csv(filepath + '\\' + "{}".format(fName), sep =',', index=False)#, quoting=csv.QUOTE_NONNUMERIC)
    return print("Saved {} to {}".format(fName,filepath))

# short find null
def null(fName):
    return df[(fName).isnull()]

# drop field (field name, file name)
def drop(fName,fileName):
    df.drop(fName, 1, inplace=True)
    df.to_csv(filepath + '\\' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Dropped {} field.\n\
Saved '{}_FIXED.txt' to file folder.".format(fName,fileName))

# drop dups in blocks.txt for CNB 
def dups():
    #df[df.block_id.duplicated()]
    df1 = df.drop_duplicates(subset = 'block_id', keep='last').reset_index(drop=True)
    return df1.info()
    
# sort sequence for stop_times.txt file
def sort():
    df.sort_values(['trip_id', 'stop_sequence'])
    return df.head(15)

# shape less than certain amount of points for shapes
def lessthan(x):
    df1 = df.groupby(['shape_id']).count()
    return df1[(df1.shape_pt_lat < x)]

# function to move copy stop_code to stop_id in the stops.txt file
def copyField(newField,origField,fileName):
    df[newField] = df[origField]
    df.to_csv(filepath + '\\' + '{}_FIXED.txt'.format(fileName), sep =',', index=False)
    return print("Copied the '{}' field to the '{}' field.\n\
Saved '{}_FIXED.txt' to file folder.".format(origField,newField,fileName))

# syntax for leading zeros (4 for example)
#df['stop_id']= df['stop_id'].apply(lambda x: '{0:0>4}'.format(x))


### work for CNB shape issues
def shapeFix(indx1,indx2,shapeName):
    x1 = int(indx1)
    x2 = int(indx2)
    global df
    df = df.drop(df.index[x1:x2]) # drop double shape pnts 
    df = df[(df.shape_id == '{}'.format(shapeName))] # create new df of just shape pnts
    df['shape_pt_sequence'] = range(1,1 + len(df)) # re-sequence shape pnts
    df_cut = df # give unique name to subset of shape pnts

    # load orig data, drop all of courthou1, append 
    load('shapes') # load orig df
    df = df[df.shape_id != '{}'.format(shapeName)] # drop all cases of shape 
    global df_all
    df_all = df.append(df_cut,ignore_index=True) # create new df and append df_cut
    return  print('use df_all')



# <span style="color:red">agency.txt</span>

In [19]:
load('agency')
df.head()

FileNotFoundError: File b'../../../mnt/shared/qa/omnitrans\\agency.txt' does not exist

# <span style="color:red">routes.txt</span>

### Required Fields: 
1. route_id- These are the routes that are shown to the public. Route ID's cannot have leading zeros, asterisks, underscore, dashes and perhaps other characters. 
2. route_type- numberic value with specific meanings:
    * 0 Tram, Streetcar, Light rail
    * 1 Subway, Metro
    * 2 Rail
    * 3 Bus
    * 4 Ferry
    * 5 Cable Car

In [17]:
# just use the funtion 'load()' to load the .txt file and see the first 5 lines ex: loadfile('routes')
load('routes')
#copyField('route_short_name','route_id','routes')




#### seattle-sc 'like' function
#df = df.loc[df['agency_id']=='EOS']
# df1['route_long_name'] = df1['route_short_name']
# df = df1
# save('routes_NEW.txt')
#df[(df.route_id==100249)]

FileNotFoundError: File b'../../mnt/shared/qa/omnitrans\\routes.txt' does not exist

# <span style="color:red">trips.txt</span>

### Required Fields:
1. route_id- Check to see if there are trips for every route. 
2. service_id- Check to see if the value of this field comes from the calendar.txt file. (below)
3. trip_id
4. direction_id- Can either be 1 or 0. Can omit if all routes are in one direction. 
5. block_id- GTFS list this as optional, NextBus requires it. Make sure there are no missig values. 
6. shape_id- GTFS list this as optional, NextBus requires it. This field links with the shapes.txt file.(below)
   


In [19]:
load('trips')
df.head()
df[(df.trip_id=='t_530303_b_none_tn_0')]

## dups work
#df[df.block_id.duplicated()]
#df1 = df.drop_duplicates(subset = 'block_id', keep='last').reset_index(drop=True)
#102638



#### seattle-sc 'like' function 
#df=df.loc[df['route_id'].isin([100340,102638])]
#tripsList = df1['trip_id'].unique().tolist()
#shapeList = df1['shape_id'].unique().tolist()
#save(df1,'trips_NEW.txt')
#df1.info()

#df['trip_id'].unique()
#df[(df.trip_id==40950103)]
# serviceList = df1['service_id'].unique().tolist()
# serviceList

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 931 entries, 0 to 930
Data columns (total 18 columns):
route_id                       931 non-null int64
service_id                     931 non-null object
trip_id                        931 non-null object
trip_short_name                4 non-null object
trip_headsign                  931 non-null object
direction_id                   931 non-null int64
block_id                       931 non-null int64
shape_id                       931 non-null object
bikes_allowed                  0 non-null float64
wheelchair_accessible          0 non-null float64
trip_type                      0 non-null float64
drt_max_travel_time            0 non-null float64
drt_avg_travel_time            0 non-null float64
drt_advance_book_min           0 non-null float64
drt_pickup_message             0 non-null float64
drt_drop_off_message           0 non-null float64
continuous_pickup_message      0 non-null float64
continuous_drop_off_message    0 non-null 

,route_id,service_id,trip_id,trip_short_name,trip_headsign,direction_id,block_id,shape_id,bikes_allowed,wheelchair_accessible,trip_type,drt_max_travel_time,drt_avg_travel_time,drt_advance_book_min,drt_pickup_message,drt_drop_off_message,continuous_pickup_message,continuous_drop_off_message
591,1041,c_16163_b_none_d_32,t_530303_b_none_tn_0,NaN,Sebastopol Shuttle,0,403,p_2794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# <span style="color:red">stops.txt</span>

### Required Fields:
1. stop_id 
2. stop_lat- Make sure none are zero, or empty.
3. stop_lon- Make sure none are zero, or empty.
    - make sure lat and long are at least five decimal places (most do six) 

In [16]:
load('stops')
df.head(25)

#### seattle-sc 'like' function
#df_stop = df.loc[df['stop_code'].isin(stopList)]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221 entries, 0 to 220
Data columns (total 15 columns):
stop_id                221 non-null int64
stop_code              221 non-null int64
platform_code          0 non-null float64
stop_name              221 non-null object
stop_desc              0 non-null float64
stop_lat               221 non-null float64
stop_lon               221 non-null float64
zone_id                0 non-null float64
stop_url               0 non-null float64
location_type          221 non-null int64
parent_station         0 non-null float64
stop_timezone          221 non-null object
position               0 non-null float64
direction              0 non-null float64
wheelchair_boarding    221 non-null int64
dtypes: float64(9), int64(4), object(2)
memory usage: 26.0+ KB


,stop_id,stop_code,platform_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,position,direction,wheelchair_boarding
0,1667,1667,NaN,Sugar Pine Point State Park,NaN,39.057527,-120.122674,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0
1,1671,1671,NaN,Hwy 89 at PDQ Market,NaN,39.068528,-120.131645,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0
2,1676,1676,NaN,Hwy 89 Across from Tahoe Ski Bowl Way,NaN,39.079344,-120.156719,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0
3,1686,1686,NaN,Hwy 89 at Sunnyside Ln,NaN,39.134198,-120.156116,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0
4,1692,1692,NaN,Tahoe City Transit Center - Arrive,NaN,39.164688,-120.144870,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0
5,1708,1708,NaN,Hwy 28 at Lake Forest Rd (West End),NaN,39.185150,-120.121466,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0
6,1710,1710,NaN,Hwy 28 at Lake Forest Rd (East End),NaN,39.189681,-120.113627,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0
7,1715,1715,NaN,Hwy 28 Across from Lardin Way,NaN,39.207226,-120.093794,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0
8,1722,1722,NaN,Hwy 28 at Stag Dr - Lakeside,NaN,39.237265,-120.062544,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0
9,1728,1728,NaN,Hwy 28 at Secline St,NaN,39.237594,-120.029587,NaN,NaN,0,NaN,America/Los_Angeles,NaN,NaN,0


# <span style="color:red">stop_times.txt</span>

### Required Fields:
1. trip_id
2. arrival_time- Should be blank for non-scheduled stops
3. depature_times- Should be blank for non-scheduled stops
4. stop_id
5. stop_sequence- Some agencies will sort this field incorrectly, putting all the "1s" first.
    - note* watch out for the timepoints field, if there is a 1, then it is an acutal stop. If there are not enough of 1s, this may cause problems. It's okay to delete this field. 

In [20]:
load('stop_times') 
#drop('timepoint','stop_times')
df.head(25)


# drop('timepoint','stop_times')

#### create new column based on conditions of other columns
# df['timeCheck'] = np.where(df['arrival_time']==df['departure_time'],'yes','no')
# df1 = df[(df.timeCheck=='no')]
# df['timeCheck'].unique()


#### seattle-sc 'like' function
# df_trip = df.loc[df['trip_id'].isin(tripsList)]
# df_trip.sort_values(['trip_id', 'stop_sequence'])
# stopList = df_trip['stop_id'].unique().tolist()
# save(df_trip,'stop_times_NEW.txt


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43562 entries, 0 to 43561
Data columns (total 20 columns):
trip_id                         43562 non-null object
arrival_time                    11062 non-null object
departure_time                  11062 non-null object
stop_id                         43562 non-null int64
stop_sequence                   43562 non-null int64
stop_headsign                   2483 non-null object
pickup_type                     43562 non-null int64
drop_off_type                   43562 non-null int64
shape_dist_traveled             43562 non-null float64
timepoint                       43562 non-null int64
start_service_area_id           0 non-null float64
end_service_area_id             0 non-null float64
start_service_area_radius       0 non-null float64
end_service_area_radius         0 non-null float64
continuous_pickup               43562 non-null int64
continuous_drop_off             43562 non-null int64
pickup_area_id                  0 non-null flo

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,start_service_area_id,end_service_area_id,start_service_area_radius,end_service_area_radius,continuous_pickup,continuous_drop_off,pickup_area_id,drop_off_area_id,pickup_service_area_radius,drop_off_service_area_radius
0,t_240748_b_6914_tn_0,15:05:00,15:05:00,757937,1,North Rohnert Park,0,0,0.000000,1,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
1,t_240748_b_6914_tn_0,NaN,NaN,757938,2,NaN,0,0,517.440667,0,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
2,t_240748_b_6914_tn_0,NaN,NaN,757940,3,NaN,0,0,844.907528,0,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,t_240748_b_6914_tn_0,15:20:00,15:20:00,757882,4,NaN,0,0,1476.964499,1,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
4,t_240748_b_6914_tn_0,NaN,NaN,757883,5,NaN,0,0,2078.401176,0,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
5,t_240748_b_6914_tn_0,NaN,NaN,757884,6,NaN,0,0,2270.351434,0,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
6,t_240748_b_6914_tn_0,NaN,NaN,757892,7,NaN,0,0,2485.869556,0,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
7,t_240748_b_6914_tn_0,NaN,NaN,757893,8,NaN,0,0,2920.436218,0,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
8,t_240748_b_6914_tn_0,NaN,NaN,757894,9,NaN,0,0,3780.728201,0,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN
9,t_240748_b_6914_tn_0,NaN,NaN,757690,10,NaN,0,0,4480.625091,0,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN,NaN


# <span style="color:red">shapes.txt</span>

### Required Fields:
1. shape_id
2. shape_pnt_lat- at least five decimal places
3. shape_pnt_lon- at least five deciaml places

In [10]:
load('shapes')
df.head(50)
# df['shape_pt_sequence'] = df['shape_pt_sequence'].astype(np.int64)
# save(df,'shapesINT.txt')
# df1 = df[(df.shape_id=='ASC_0_var0')]
# save(df1,'ASC_0_var0.csv')


#### seattle-sc 'like' function
# df_shape = df.loc[df['shape_id'].isin(shapeList)]
# save(df_shape,'shapes_NEW.txt')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28476 entries, 0 to 28475
Data columns (total 5 columns):
shape_id               28476 non-null object
shape_pt_lat           28476 non-null float64
shape_pt_lon           28476 non-null float64
shape_pt_sequence      28476 non-null int64
shape_dist_traveled    28476 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 1.1+ MB


,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,p_180473,39.164738,-120.144976,1,0.000000
1,p_180473,39.164997,-120.144613,2,42.576471
2,p_180473,39.165039,-120.144505,3,52.966806
3,p_180473,39.165035,-120.144429,4,59.543409
4,p_180473,39.164993,-120.144377,5,66.058215
5,p_180473,39.164939,-120.144349,6,72.473737
6,p_180473,39.164897,-120.144358,7,77.243608
7,p_180473,39.164509,-120.144874,8,139.251214
8,p_180473,39.164468,-120.145202,9,167.912834
9,p_180473,39.164190,-120.145238,10,199.015707


# <span style="color:red">calendar.txt</span>

### Required:
Run the calendar script if more than one service record is defined for the day of the week.
* If there is more than one service defined for the week, then run the calendar.py script.


In [17]:
load('calendar')
df.head(35)


#### seattle-sc 'like' function
# df_cal = df.loc[df['service_id'].isin(serviceList)]
# df_cal

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 11 columns):
service_id      15 non-null object
service_name    15 non-null object
monday          15 non-null int64
tuesday         15 non-null int64
wednesday       15 non-null int64
thursday        15 non-null int64
friday          15 non-null int64
saturday        15 non-null int64
sunday          15 non-null int64
start_date      15 non-null int64
end_date        15 non-null int64
dtypes: int64(9), object(2)
memory usage: 1.4+ KB


,service_id,service_name,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,c_6380_b_6914_d_4,School Trippers - Beginning August 14 (Wednesd...,0,0,1,0,0,0,0,20180814,20190601
1,c_6380_b_6914_d_31,School Trippers - Beginning August 14 (Monday-...,1,1,1,1,1,0,0,20180814,20190601
2,c_6380_b_6914_d_27,School Trippers - Beginning August 14,1,1,0,1,1,0,0,20180814,20190601
3,c_16166_b_17859_d_32,Route 32 - Ending December 11 (Saturday only),0,0,0,0,0,1,0,20180812,20181211
4,c_16166_b_17859_d_31,Route 32 - Ending December 11 (Monday-Friday),1,1,1,1,1,0,0,20180812,20181211
5,c_16164_b_17854_d_32,Route 32 - Beginning December 12 (Saturday only),0,0,0,0,0,1,0,20181212,20190601
6,c_16164_b_17854_d_31,Route 32 - Beginning December 12 (Monday-Friday),1,1,1,1,1,0,0,20181212,20190601
7,c_16163_b_17846_d_96,Year Round - Beginning December 9 (Weekend),0,0,0,0,0,1,1,20181209,20190601
8,c_16163_b_17846_d_32,Year Round - Beginning December 9 (Saturday only),0,0,0,0,0,1,0,20181209,20190601
9,c_16163_b_17846_d_31,Year Round - Beginning December 9 (Monday-Friday),1,1,1,1,1,0,0,20181209,20190601


# <span style="color:red">calendar_dates.txt</span>

### Required:
This is an optional file, but is used to mark special holidays. Make sure the dates make sense.

In [14]:
load('calendar_dates')
df.head(50)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 4 columns):
service_id        21 non-null object
date              21 non-null int64
holiday_name      21 non-null object
exception_type    21 non-null int64
dtypes: int64(2), object(2)
memory usage: 752.0+ bytes


,service_id,date,holiday_name,exception_type
0,c_6380_b_6914_d_31,20181231,New Years Eve,2
1,c_6380_b_6914_d_31,20181225,Christmas Day,2
2,c_6380_b_6914_d_31,20181224,Christmas Eve,2
3,c_6380_b_6914_d_27,20181231,New Years Eve,2
4,c_6380_b_6914_d_27,20181225,Christmas Day,2
5,c_6380_b_6914_d_27,20181224,Christmas Eve,2
6,c_16166_b_17859_d_31,20181225,Christmas Day,2
7,c_16166_b_17859_d_31,20181224,Christmas Eve,2
8,c_16164_b_none_d_32,20181231,New Years Eve,1
9,c_16164_b_17854_d_31,20181231,New Years Eve,2
